<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# # Project 2 : Ames Housing Project Suggestions - Final Report

## Part 1: Imports and Read

In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt

In [1]:
# call only one url
url='https://api.pushshift.io/reddit/search/submission?subreddit=boardgames'

In [ ]:
push_res = requests.get(url)

## Part2: Output

## Part4: Presentation 

Please refer to Ames Presentation.pdf for the final presentation.
